In [22]:
#Importing Libraries

import numpy as np #To handle Mathematical calculations
import matplotlib.pyplot as plt #To plot charts 
%matplotlib inline
import pandas as pd #To import and manage datasets
import glob
import os
import warnings
pd.set_option('display.max_columns', None)  
# pd.set_option('display.max_rows', None)  

#Disable warning messages
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [23]:
#Load preprocessed dataset
combinedPlayerDataframe = pd.read_csv("../Pre_Processed_Datasets/Preprocessed Player Data.csv") 

In [24]:
combinedPlayerDataframe.head()

,Name,Team,Age,Position,Apps,Minutes,Total goals,Total Assists,Shots per game,Key passes per game,Dribbles per game,Fouled per game,Offsides per game,Dispossessed per game,Bad control per game,Rating,League,Season,Tackles per game,Interceptions per game,Fouls per game,Clearances per game,Dribbled past per game,Outfield blocks per game,Own goals,Passes per game,Pass success percentage,Crosses per game,Long balls per game,Through balls per game,Total Goals,Yellow cards,Red cards,Aerials won per game,Man of the match
0,Arjen Robben,Bayern Munich,26,Midfielder,18,1779,16,6,3.4,1.8,4.0,2.2,0.3,2.1,0.0,8.18,Bundesliga,2009,0.6,0.5,0.9,0.0,0.7,0.0,0,34.3,80.2,1.5,2.0,0.2,16,1,0,0.4,8
1,Stefan Kießling,Bayer Leverkusen,26,Forward,33,2924,21,5,3.0,1.6,1.1,3.1,1.8,2.2,0.0,7.79,Bundesliga,2009,1.2,0.7,2.5,0.3,0.5,0.1,0,32.5,72.4,0.2,0.4,0.1,21,3,0,2.1,6
2,Zvjezdan Misimovic,Wolfsburg,28,Attacking Midfielder,31,2768,10,13,2.4,3.3,1.5,2.1,0.3,3.1,0.0,7.74,Bundesliga,2009,1.9,0.6,1.5,0.2,1.2,0.1,0,51.1,77.5,2.5,5.7,0.5,10,7,0,0.2,8
3,Edin Dzeko,Wolfsburg,24,Forward,33,3003,22,7,4.5,1.1,2.0,1.7,1.6,2.9,0.0,7.73,Bundesliga,2009,0.5,0.1,1.9,0.9,0.2,0.1,0,19.5,70.7,0.2,1.1,0.1,22,4,0,0.7,6
4,Claudio Pizarro,Werder Bremen,31,Attacking Midfielder,23,2130,16,2,2.6,1.3,1.2,1.2,1.4,3.1,0.0,7.63,Bundesliga,2009,1.0,0.7,2.0,0.7,0.4,0.2,0,24.2,75.5,0.1,0.8,0.0,16,4,0,0.4,2


In [25]:
#Load forecasted rating dataset
forecastedPlayerDataframe = pd.read_csv("../Pre_Processed_Datasets/Forecasted Ratings 2019-2022.csv") 

In [26]:
forecastedPlayerDataframe.head()

,Name,Season,Rating,Age,Mean,Growth
0,Aaron Hughes,2009,6.95,30,6.880000,-0.20
1,Aaron Hunt,2009,7.15,23,7.023750,-0.36
2,Aaron Lennon,2009,7.33,23,6.806667,-0.95
3,Aaron Mokoena,2009,6.56,29,6.560000,0.00
4,Abdelhamid El Kaoutari,2009,6.70,20,6.866667,-0.20


### Handling young player data (Age: 23-28, Season 2018)

In [27]:
#Creating a seperate DF for players who are between the age of 23-28 in 2018

youngPlayers2018Df = forecastedPlayerDataframe.loc[(forecastedPlayerDataframe.Season == 2018) 
                                                   & ((forecastedPlayerDataframe.Age > 22) 
                                                       & (forecastedPlayerDataframe.Age < 29))]

In [28]:
#Storing the names of the young players on to list

youngPlayerNames = youngPlayers2018Df.Name.tolist()

In [29]:
#Creating a seperate dictionary entry for every player
#Creating a new row of data for every player with the mean of all their stats
#Rating has been replaced by the forecasted rating average

young_players_ = {}

for name in youngPlayerNames:
    playerMeanRating = forecastedPlayerDataframe.loc[(forecastedPlayerDataframe.Name == name)]['Rating'].mean()
    playerDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == name)]
    playerDf = playerDf.sort_values(by='Age', ascending=True)
    playerDf.loc['average'] =  playerDf.mean()
    playerDf.loc['average', 'Name'] = name
    playerDf.loc['average', 'Team'] = playerDf['Team'][playerDf.index[-2]]
    playerDf.loc['average', 'Position'] = playerDf['Position'][playerDf.index[-2]]
    playerDf.loc['average', 'League'] = playerDf['League'][playerDf.index[-2]]
    playerDf.loc['average', 'Rating'] = playerMeanRating
    playerDf = playerDf.iloc[-1:]
    young_players_[name] = playerDf

In [30]:
young_players_['Leroy Sané']

,Name,Team,Age,Position,Apps,Minutes,Total goals,Total Assists,Shots per game,Key passes per game,Dribbles per game,Fouled per game,Offsides per game,Dispossessed per game,Bad control per game,Rating,League,Season,Tackles per game,Interceptions per game,Fouls per game,Clearances per game,Dribbled past per game,Outfield blocks per game,Own goals,Passes per game,Pass success percentage,Crosses per game,Long balls per game,Through balls per game,Total Goals,Yellow cards,Red cards,Aerials won per game,Man of the match
average,Leroy Sané,Manchester City,21.5,Midfielder,22.75,2098.0,8.25,8.5,1.75,1.3,2.325,0.825,0.4,1.725,2.325,7.24375,PremierLeague,2016.5,1.175,0.525,0.65,0.15,0.75,0.025,0.0,26.7,81.85,0.375,0.325,0.05,8.25,3.25,0.0,0.6,1.75


In [31]:
#Grouping youg player data into seperate position based dictionary entries

postionYoungPlayers_ = {}

Midfielders = []
Forwards = []
Attacking_Midfielders = []
Defenders = []
Goalkeepers = []
Defensive_Midfielders = []

for name in young_players_:
    position = young_players_[name].Position.tolist()[0]
    if position == 'Midfielder':
        Midfielders.append(young_players_[name])
    if position == 'Forward':
        Forwards.append(young_players_[name])
    if position == 'Attacking Midfielder':
        Attacking_Midfielders.append(young_players_[name])
    if position == 'Defender':
        Defenders.append(young_players_[name])
    if position == 'Goalkeeper':
        Goalkeepers.append(young_players_[name])
    if position == 'Defensive Midfielder':
        Defensive_Midfielders.append(young_players_[name])
    
postionYoungPlayers_['Midfielder']  = pd.concat(Midfielders)
postionYoungPlayers_['Forward'] = pd.concat(Forwards)
postionYoungPlayers_['Attacking Midfielder'] = pd.concat(Attacking_Midfielders)
postionYoungPlayers_['Defender'] = pd.concat(Defenders)
postionYoungPlayers_['Goalkeeper'] = pd.concat(Goalkeepers)
postionYoungPlayers_['Defensive Midfielder'] = pd.concat(Defensive_Midfielders)

In [32]:
postionYoungPlayers_['Defensive Midfielder'].head()

,Name,Team,Age,Position,Apps,Minutes,Total goals,Total Assists,Shots per game,Key passes per game,Dribbles per game,Fouled per game,Offsides per game,Dispossessed per game,Bad control per game,Rating,League,Season,Tackles per game,Interceptions per game,Fouls per game,Clearances per game,Dribbled past per game,Outfield blocks per game,Own goals,Passes per game,Pass success percentage,Crosses per game,Long balls per game,Through balls per game,Total Goals,Yellow cards,Red cards,Aerials won per game,Man of the match
average,Abdoulaye Touré,Nantes,24.500000,Defensive Midfielder,30.500000,2781.500000,2.000000,1.000000,1.350000,0.600000,1.250000,1.450000,0.000000,1.250000,1.250000,6.865000,Ligue1,2017.500000,1.750000,1.550000,1.150000,1.000000,1.150000,0.300000,0.000000,38.600000,79.900000,0.150000,1.900000,0.050000,2.000000,3.500000,0.500000,1.650000,2.000000
average,Adrien Rabiot,Paris Saint-Germain,21.000000,Defensive Midfielder,16.000000,1473.571429,2.000000,1.857143,0.757143,0.514286,1.171429,1.071429,0.014286,1.014286,0.871429,7.189091,Ligue1,2015.000000,2.257143,1.514286,1.242857,0.842857,0.714286,0.242857,0.142857,54.842857,89.200000,0.142857,2.200000,0.142857,2.000000,2.000000,0.142857,1.385714,1.428571
average,Afriyie Acquah,Empoli,23.428571,Defensive Midfielder,14.857143,1297.285714,1.142857,0.714286,0.857143,0.628571,1.057143,1.214286,0.028571,1.128571,1.000000,6.550000,SerieA,2014.428571,1.385714,0.957143,1.742857,0.642857,0.514286,0.071429,0.000000,29.585714,81.442857,0.242857,1.314286,0.057143,1.142857,5.142857,0.714286,0.700000,0.285714
average,Allan,Napoli,25.000000,Defensive Midfielder,29.571429,2657.571429,1.428571,4.571429,0.642857,1.014286,1.785714,1.771429,0.000000,1.528571,1.171429,7.180000,SerieA,2015.000000,3.500000,1.242857,1.300000,0.757143,1.928571,0.257143,0.000000,48.642857,85.357143,0.157143,2.542857,0.100000,1.428571,5.714286,0.142857,0.457143,1.571429
average,Andrei Girotto,Nantes,26.500000,Defensive Midfielder,18.000000,1500.000000,1.500000,0.500000,1.100000,0.400000,0.250000,1.000000,0.000000,0.400000,0.600000,6.738333,Ligue1,2017.500000,2.600000,1.050000,1.550000,1.850000,1.200000,0.300000,0.000000,32.800000,77.850000,0.100000,1.950000,0.000000,1.500000,6.000000,1.500000,2.350000,0.500000


### Predicting replacements for older players

In [33]:
#Choose club

club_name = 'Real Madrid'

In [34]:
#Creating a sperate DF for older players

olderPlayersDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Team == club_name) 
                                            & (combinedPlayerDataframe.Season == 2018) 
                                            & (combinedPlayerDataframe.Age > 32)]

In [35]:
#Creating a seperate dictionary to store the average stat data of the best years (24-29) of the older players

older_players_ = {}

for index, row in olderPlayersDf.iterrows():
    
    currentDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == row['Name']) 
                                            & (combinedPlayerDataframe.Age > 23)  
                                            & (combinedPlayerDataframe.Age < 30)]
    currentDf = currentDf.sort_values(by='Age', ascending=True)
    currentDf.loc['average'] =  currentDf.mean()
    currentDf.loc['average', 'Name'] = row['Name']
    currentDf.loc['average', 'Team'] = currentDf['Team'][currentDf.index[-2]]
    currentDf.loc['average', 'Position'] = currentDf['Position'][currentDf.index[-2]]
    currentDf.loc['average', 'League'] = currentDf['League'][currentDf.index[-2]]
    currentDf = currentDf.iloc[-1:]
    older_players_[row['Name']] = currentDf

In [53]:
#Recommending young replacements within the age of (23-28)

li = []

old_player_ = {}

for name in older_players_:
    position = older_players_[name].Position.tolist()[0]
    li.append(older_players_[name])
    youngDf = postionYoungPlayers_[position]
    youngDf = youngDf[youngDf.Team != club_name]
    li.append(youngDf)
    
    combinedDf = pd.concat(li)
    
    stats = combinedDf.copy(deep=True)
    stats.drop(['Name', 'Team', 'League', 'Position'], axis=1, inplace=True)
    
    stats_with_name = stats
    stats = stats.dropna()
    stats_with_name = stats_with_name.dropna()
    stats_with_name["Name"] = combinedDf["Name"]
    
    stats_with_name = stats_with_name.reset_index(drop=True)
    
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import NearestNeighbors
    from sklearn.exceptions import DataConversionWarning

    import warnings
    warnings.filterwarnings(action="ignore",category=DataConversionWarning)
    
    scaler = StandardScaler()
    
    X = scaler.fit_transform(stats)
    
    #Limiting the number of similar profiles(clusters) to be found to 3. (4 in total with the current player being one)
    knn = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)

    player_indexes = knn.kneighbors(X)[1]
    
    print("")
    print("Older Player: " + name)
    
    old_player_[name] = get_player_df(name)
    
    index = stats_with_name[stats_with_name["Name"]==name].index.tolist()[0]
    count = 1
    for i in player_indexes[index][1:]:
            print('Option ' + str(count) + ': ' + stats_with_name.iloc[i]["Name"])
            
            old_player_[name] = old_player_[name].append(get_player_df(stats_with_name.iloc[i]["Name"]))
            
            count += 1

    
    li=[]
# print(old_player_['Luka Modric'])


Older Player: Luka Modric
Option 1: Ilkay Gündogan
Option 2: Thiago Alcántara
Option 3: Kevin Kampl

Older Player: Sergio Ramos
Option 1: Íñigo Martínez
Option 2: Salif Sané
Option 3: Aymeric Laporte


### Predicting replacements for active underperforming players

In [37]:
#Choose club

club_name = 'Real Madrid'

In [38]:
activeClubPlayers = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Team == club_name) 
                                            & (combinedPlayerDataframe.Season == 2018) 
                                            & (combinedPlayerDataframe.Age > 27)
                                            & (combinedPlayerDataframe.Age < 33)]

In [39]:
activePlayerNames = activeClubPlayers.Name.tolist()

In [40]:
active_players_ = {}

for name in activePlayerNames:
    previouseFormDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == name)
                                               & (combinedPlayerDataframe.Age > 23)
                                               & (combinedPlayerDataframe.Age < 28)]
    
    recentFormDf = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == name)
                                               & (combinedPlayerDataframe.Age > 27)
                                               & (combinedPlayerDataframe.Age < 33)]
    
    previouseRatingAvg = previouseFormDf.Rating.mean()
    recentRatingAvg = recentFormDf.Rating.mean()
    decline = (previouseRatingAvg-recentRatingAvg)
    
    if (recentRatingAvg < 8) & (decline >= 0.3) :
#         print('Name: ' + name)
#         print('Previouse Avg: ' + "%.2f" % previouseRatingAvg)
#         print('Current Avg: ' + "%.2f" % recentRatingAvg)
#         print('decline: ' + "%.2f" % decline)
#         print("")
        
        previouseFormDf = previouseFormDf.sort_values(by='Age', ascending=True)
        previouseFormDf.loc['average'] =  previouseFormDf.mean()
        previouseFormDf.loc['average', 'Name'] = name
        previouseFormDf.loc['average', 'Team'] = previouseFormDf['Team'][previouseFormDf.index[-2]]
        previouseFormDf.loc['average', 'Position'] = previouseFormDf['Position'][previouseFormDf.index[-2]]
        previouseFormDf.loc['average', 'League'] = previouseFormDf['League'][previouseFormDf.index[-2]]
        previouseFormDf = previouseFormDf.iloc[-1:]
        active_players_[name] = previouseFormDf
    
    elif (recentRatingAvg < 7) & (decline >= 0.2):
#         print('Name: ' + name)
#         print('Previouse Avg: ' + "%.2f" % previouseRatingAvg)
#         print('Current Avg: ' + "%.2f" % recentRatingAvg)
#         print('decline: ' + "%.2f" % decline)
#         print("")
        
        previouseFormDf = previouseFormDf.sort_values(by='Age', ascending=True)
        previouseFormDf.loc['average'] =  previouseFormDf.mean()
        previouseFormDf.loc['average', 'Name'] = name
        previouseFormDf.loc['average', 'Team'] = previouseFormDf['Team'][previouseFormDf.index[-2]]
        previouseFormDf.loc['average', 'Position'] = previouseFormDf['Position'][previouseFormDf.index[-2]]
        previouseFormDf.loc['average', 'League'] = previouseFormDf['League'][previouseFormDf.index[-2]]
        previouseFormDf = previouseFormDf.iloc[-1:]
        active_players_[name] = previouseFormDf

In [55]:
#Recommending young replacements within the age of (23-28)

li = []

active_player_ = {}

for name in active_players_:
    position = active_players_[name].Position.tolist()[0]
    li.append(active_players_[name])
    youngDf = postionYoungPlayers_[position]
    youngDf = youngDf[youngDf.Team != club_name]
    li.append(youngDf)
    
    combinedDf = pd.concat(li)
    
    stats = combinedDf.copy(deep=True)
    stats.drop(['Name', 'Team', 'League', 'Position'], axis=1, inplace=True)
    
    stats_with_name = stats
    stats = stats.dropna()
    stats_with_name = stats_with_name.dropna()
    stats_with_name["Name"] = combinedDf["Name"]
    
    stats_with_name = stats_with_name.reset_index(drop=True)
    
    from sklearn.preprocessing import StandardScaler
    from sklearn.neighbors import NearestNeighbors
    from sklearn.exceptions import DataConversionWarning

    import warnings
    warnings.filterwarnings(action="ignore",category=DataConversionWarning)
    
    scaler = StandardScaler()
    
    X = scaler.fit_transform(stats)
    
    #Limiting the number of similar profiles(clusters) to be found to 3. (4 in total with the current player being one)
    knn = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)

    player_indexes = knn.kneighbors(X)[1]
    
    print("")
    print("Active Underperfoming Player: " + name)
    
    active_player_[name] = get_player_df(name)
    
    index = stats_with_name[stats_with_name["Name"]==name].index.tolist()[0]
    count = 1
    for i in player_indexes[index][1:]:
            print('Option ' + str(count) + ': ' + stats_with_name.iloc[i]["Name"])
            
            active_player_[name] = active_player_[name].append(get_player_df(stats_with_name.iloc[i]["Name"]))
            
            count += 1

    
    li=[]
# print(active_player_)


Active Underperfoming Player: Gareth Bale
Option 1: Alexandre Lacazette
Option 2: Quincy Promes
Option 3: Son Heung-Min

Active Underperfoming Player: Keylor Navas
Option 1: Benjamin Lecomte
Option 2: Baptiste Reynet
Option 3: Marco Sportiello


In [42]:
# saving the model
import pickle
with open('../Models/model_replacements_knn.pkl','wb') as file:
    pickle.dump(knn, file)

In [45]:
def get_player_df(name):
    player_df  = combinedPlayerDataframe.loc[(combinedPlayerDataframe.Name == name) & (combinedPlayerDataframe.Season == 2018)]
    return player_df